In [1]:
import psycopg2
import sqlalchemy
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy.stats import expon
%matplotlib inline

from sqlalchemy import create_engine

postgres_str = ('postgresql://localhost/crossvalidated')
cnx = create_engine(postgres_str)

In [2]:
import pandas as pd
import numpy as np

### Query reputation for comparison

In [38]:
reputation = pd.read_sql_query("SELECT Id, Reputation FROM Users WHERE Id<50", cnx) 

In [40]:
reputation.head(50)

,id,reputation
0,-1,1
1,2,101
2,3,101
3,4,101
4,5,9603
5,6,597
6,7,663
7,8,10169
8,10,131
9,11,136


## Compute the reputation for each point of time

In [3]:
def compute_factor(row):
    if row["posttypeid"]==1 and row["votetypeid"]==2: # 
        factor = 5
    elif row["posttypeid"]==1 and row["votetypeid"]==3:
        factor = -5
    elif row["posttypeid"]==2 and row["votetypeid"]==2:
        factor = 10
    elif row["posttypeid"]==2 and row["votetypeid"]==3:
        factor = -10
    elif row["posttypeid"]==2 and row["votetypeid"]==1: # accepted answer
        factor = 15
    elif row["votetypeid"]==8:
        factor = row["bountyamount"]
    else: 
        factor = 0
    res = factor*row["postid"]
    return res

DATE = '2016-03-14 15:15:10.813000'
        
reputation = pd.read_sql_query("SELECT * FROM Votes LEFT JOIN (SELECT Id, PostTypeId, OwnerUserId FROM Posts) b ON Votes.Id=b.Id WHERE CreationDate<'{}'".format(DATE), cnx) 
grouped = reputation.groupby(["owneruserid", "posttypeid", "votetypeid"]).agg({"postid": "count", "bountyamount":"sum"})
grouped = grouped.reset_index()
grouped["score"] = grouped.apply(compute_factor, axis=1)
# grouped["factor"] = grouped[["posttypeid", "votetypeid"]].apply(lambda x, y: 5 if x==1 and y==2 else 0, axis=1)
out = grouped.groupby(["owneruserid"]).agg({"score":"sum"})

In [5]:
out.head(20)

,score
owneruserid,
-1.0,200.0
5.0,800.0
6.0,65.0
7.0,5.0
8.0,940.0
10.0,0.0
11.0,45.0
12.0,10.0
13.0,55.0
